In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torchdiffeq import odeint
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler

In [2]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [3]:
# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

In [4]:
# Load the data
complete_data = pd.read_csv('../data/raw/uav_hugging_face.csv')
incomplete_data = pd.read_csv('../data/raw/uav_hugging_face.csv')

In [5]:
# Extract time and features
complete_time = complete_data['timestamp'].values
complete_features = complete_data[['tx', 'ty', 'tz']].values


In [6]:
incomplete_time = incomplete_data['timestamp'].values
incomplete_features = incomplete_data[['tx', 'ty', 'tz']].values


In [7]:
# Normalize the time to [0, 1]
time_scaler = MinMaxScaler()
complete_time_norm = time_scaler.fit_transform(complete_time.reshape(-1, 1)).flatten()


In [8]:
# Normalize features
feature_scaler = MinMaxScaler()
complete_features_norm = feature_scaler.fit_transform(complete_features)
incomplete_features_norm = feature_scaler.transform(incomplete_features)


In [9]:
# Create a mask for observed time points in the incomplete dataset
observed_mask = np.zeros(len(complete_time))
for t in incomplete_time:
    idx = np.where(complete_time == t)[0]
    if len(idx) > 0:
        observed_mask[idx[0]] = 1

In [10]:
# Convert to PyTorch tensors
complete_time_tensor = torch.tensor(complete_time_norm, dtype=torch.float32)
complete_features_tensor = torch.tensor(complete_features_norm, dtype=torch.float32)
incomplete_time_tensor = torch.tensor(time_scaler.transform(incomplete_time.reshape(-1, 1)).flatten(), dtype=torch.float32)
incomplete_features_tensor = torch.tensor(incomplete_features_norm, dtype=torch.float32)
observed_mask_tensor = torch.tensor(observed_mask, dtype=torch.float32)


In [11]:
# Define the Latent ODE model components
class LatentODEFunc(nn.Module):
    """ODE function for the latent dynamics"""
    def __init__(self, latent_dim, hidden_dim):
        super(LatentODEFunc, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, latent_dim)
        )
        
    def forward(self, t, x):
        """Time derivative of the latent state"""
        return self.net(x)

In [12]:
class Encoder(nn.Module):
    """Encodes the observed data into initial latent state"""
    def __init__(self, input_dim, hidden_dim, latent_dim):
        super(Encoder, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, latent_dim * 2)  # For mean and logvar
        
    def forward(self, x, time):
        """
        x: [batch_size, seq_len, input_dim]
        time: [batch_size, seq_len]
        """
        # Sort by time
        _, indices = torch.sort(time, dim=1)
        batch_size, seq_len = x.size(0), x.size(1)
        sorted_indices = indices.unsqueeze(-1).expand(-1, -1, x.size(-1))
        x_sorted = torch.gather(x, 1, sorted_indices)
        
        h, _ = self.lstm(x_sorted)
        h = h[:, -1, :]  # Take the last hidden state
        
        params = self.linear(h)
        mean, logvar = torch.chunk(params, 2, dim=1)
        
        # Reparameterization trick
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        z0 = mean + eps * std
        
        return z0, mean, logvar


In [13]:

class Decoder(nn.Module):
    """Decodes the latent state into observed data"""
    def __init__(self, latent_dim, hidden_dim, output_dim):
        super(Decoder, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )
        
    def forward(self, z):
        """
        z: [batch_size, seq_len, latent_dim]
        """
        return self.net(z)


In [14]:
class LatentODE(nn.Module):
    """Complete Latent ODE model"""
    def __init__(self, input_dim, latent_dim, hidden_dim, device):
        super(LatentODE, self).__init__()
        self.encoder = Encoder(input_dim, hidden_dim, latent_dim)
        self.func = LatentODEFunc(latent_dim, hidden_dim)
        self.decoder = Decoder(latent_dim, hidden_dim, input_dim)
        self.latent_dim = latent_dim
        self.device = device
        
    def forward(self, x, time_points, target_time_points):
        """
        x: [batch_size, seq_len, input_dim]
        time_points: [batch_size, seq_len]
        target_time_points: times to evaluate the ODE at
        """
        batch_size = x.size(0)
        
        # Get initial latent state
        z0, mean, logvar = self.encoder(x, time_points)
        
        # Integrate the ODE
        target_time_points = target_time_points.to(self.device)
        z = odeint(self.func, z0, target_time_points)
        z = z.permute(1, 0, 2)  # [batch_size, seq_len, latent_dim]
        
        # Decode
        x_pred = self.decoder(z)
        
        return x_pred, mean, logvar


In [15]:
# Model hyperparameters
input_dim = 3  # tx, ty, tz
latent_dim = 4
hidden_dim = 64
batch_size = 176
epochs = 1000
lr = 1e-3

In [16]:
# Reshape for batched training (we'll use a batch size of 1 since we have one trajectory)
complete_dataset = TensorDataset(
    complete_features_tensor.unsqueeze(0), 
    complete_time_tensor.unsqueeze(0)
)
complete_loader = DataLoader(complete_dataset, batch_size=1, shuffle=False)


In [17]:
# Create and train the model
model = LatentODE(input_dim, latent_dim, hidden_dim, device).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)


In [18]:
def loss_fn(pred, target, mean, logvar, mask=None):
    """
    Loss function with reconstruction loss and KL divergence
    """
    if mask is None:
        mask = torch.ones_like(target)
    
    # MSE reconstruction loss
    recon_loss = torch.sum(mask.unsqueeze(-1) * (pred - target)**2) / torch.sum(mask.unsqueeze(-1))
    
    # KL divergence
    kl_loss = -0.5 * torch.mean(1 + logvar - mean.pow(2) - logvar.exp())
    
    return recon_loss + 0.1 * kl_loss, recon_loss, kl_loss


In [19]:
# Training loop
print("Starting training...")
losses = []

for epoch in range(epochs):
    model.train()
    epoch_loss = 0.0
    
    for batch_idx, (x_batch, t_batch) in enumerate(complete_loader):
        x_batch = x_batch.to(device)
        t_batch = t_batch.to(device)
        
        optimizer.zero_grad()
        
        # Forward pass
        x_pred, mean, logvar = model(x_batch, t_batch, complete_time_tensor.to(device))
        
        # Compute loss
        loss, recon_loss, kl_loss = loss_fn(x_pred, x_batch, mean, logvar, observed_mask_tensor.to(device))
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
    
    avg_loss = epoch_loss / len(complete_loader)
    losses.append(avg_loss)
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.6f}")

print("Training completed!")

Starting training...
Epoch 10/1000, Loss: 0.979713
Epoch 20/1000, Loss: 0.607970
Epoch 30/1000, Loss: 0.281398
Epoch 40/1000, Loss: 0.307383
Epoch 50/1000, Loss: 0.300703
Epoch 60/1000, Loss: 0.280158
Epoch 70/1000, Loss: 0.314379
Epoch 80/1000, Loss: 0.266520
Epoch 90/1000, Loss: 0.291454
Epoch 100/1000, Loss: 0.264280
Epoch 110/1000, Loss: 0.286660
Epoch 120/1000, Loss: 0.266207
Epoch 130/1000, Loss: 0.276017
Epoch 140/1000, Loss: 0.285599
Epoch 150/1000, Loss: 0.283290
Epoch 160/1000, Loss: 0.309955
Epoch 170/1000, Loss: 0.280468
Epoch 180/1000, Loss: 0.268880
Epoch 190/1000, Loss: 0.261663
Epoch 200/1000, Loss: 0.261596
Epoch 210/1000, Loss: 0.259944
Epoch 220/1000, Loss: 0.260344
Epoch 230/1000, Loss: 0.295365
Epoch 240/1000, Loss: 0.263374
Epoch 250/1000, Loss: 0.257621
Epoch 260/1000, Loss: 0.262226
Epoch 270/1000, Loss: 0.265967
Epoch 280/1000, Loss: 0.261428
Epoch 290/1000, Loss: 0.259622
Epoch 300/1000, Loss: 0.288579
Epoch 310/1000, Loss: 0.263779
Epoch 320/1000, Loss: 0.262

In [20]:
# Evaluation and visualization
model.eval()
with torch.no_grad():
    x_complete = complete_features_tensor.unsqueeze(0).to(device)
    t_complete = complete_time_tensor.unsqueeze(0).to(device)
    
    # Get predictions for all time points
    x_pred, _, _ = model(x_complete, t_complete, complete_time_tensor.to(device))
    
    # Move predictions back to CPU and convert to numpy
    x_pred = x_pred.squeeze(0).cpu().numpy()
    
    # Inverse transform to get the original scale
    x_pred_original = feature_scaler.inverse_transform(x_pred)


In [21]:
# Compare the reconstructed trajectory with the original
plt.figure(figsize=(15, 10))
features = ['tx', 'ty', 'tz']

for i in range(3):
    plt.subplot(3, 1, i+1)
    
    # Original complete data
    plt.plot(complete_time, complete_features[:, i], 'b-', label='Original Complete')
    
    # Incomplete data points
    plt.plot(incomplete_time, incomplete_features[:, i], 'ro', markersize=4, label='Observed (Incomplete)')
    
    # Reconstructed trajectory
    plt.plot(complete_time, x_pred_original[:, i], 'g--', label='Reconstructed')
    
    plt.title(f'Feature: {features[i]}')
    plt.xlabel('Time')
    plt.ylabel(features[i])
    plt.legend()

plt.tight_layout()
plt.savefig('../output/4000_epoch/uav_trajectory_reconstruction.png')
plt.close()

In [22]:
# Calculate reconstruction error
# First, find indices that exist in complete dataset but not in incomplete
missing_indices = np.where(observed_mask == 0)[0]
missing_original = complete_features[missing_indices]
missing_reconstructed = x_pred_original[missing_indices]

mse = np.mean((missing_original - missing_reconstructed) ** 2)
mae = np.mean(np.abs(missing_original - missing_reconstructed))

print(f"Missing points reconstruction - MSE: {mse:.6f}, MAE: {mae:.6f}")


Missing points reconstruction - MSE: nan, MAE: nan


/home/chicken/Pytorch/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/chicken/Pytorch/lib/python3.10/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [23]:
# Calculate error metrics for each dimension
for i in range(3):
    mse_dim = np.mean((missing_original[:, i] - missing_reconstructed[:, i]) ** 2)
    mae_dim = np.mean(np.abs(missing_original[:, i] - missing_reconstructed[:, i]))
    print(f"Dimension {features[i]} - MSE: {mse_dim:.6f}, MAE: {mae_dim:.6f}")


Dimension tx - MSE: nan, MAE: nan
Dimension ty - MSE: nan, MAE: nan
Dimension tz - MSE: nan, MAE: nan


In [24]:
# Create a more detailed comparison plot for missing points
plt.figure(figsize=(15, 10))
for i in range(3):
    plt.subplot(3, 1, i+1)
    
    # Original values at missing points
    plt.plot(complete_time[missing_indices], missing_original[:, i], 'bx', markersize=6, label='Original (Missing)')
    
    # Reconstructed values at missing points
    plt.plot(complete_time[missing_indices], missing_reconstructed[:, i], 'go', markersize=4, label='Reconstructed')
    
    plt.title(f'Reconstruction of Missing Points: {features[i]}')
    plt.xlabel('Time')
    plt.ylabel(features[i])
    plt.legend()

plt.tight_layout()
plt.savefig('../output/4000_epoch/missing_points_comparison.png')
plt.close()

In [25]:
# Plot training loss
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(losses) + 1), losses)
plt.title('Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True)
plt.savefig('../output/4000_epoch/training_loss.png')
plt.close()


In [26]:
# 3D trajectory visualization
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Original complete trajectory
ax.plot3D(complete_features[:, 0], complete_features[:, 1], complete_features[:, 2], 
         'b-', linewidth=1, label='Original Complete')

# Observed points
ax.scatter(incomplete_features[:, 0], incomplete_features[:, 1], incomplete_features[:, 2], 
          c='r', s=30, label='Observed Points')

# Reconstructed trajectory
ax.plot3D(x_pred_original[:, 0], x_pred_original[:, 1], x_pred_original[:, 2], 
         'g--', linewidth=2, label='Reconstructed')

ax.set_xlabel('tx')
ax.set_ylabel('ty')
ax.set_zlabel('tz')
ax.set_title('UAV 3D Trajectory Reconstruction')
ax.legend()

plt.tight_layout()
plt.savefig('../output/4000_epoch/uav_3d_trajectory.png')
plt.close()

In [27]:
print("Visualization completed! Check the output images for results.")

Visualization completed! Check the output images for results.


In [28]:
# Save the reconstructed trajectory to CSV
reconstructed_df = pd.DataFrame({
    'timestamp': complete_time,
    'tx': x_pred_original[:, 0],
    'ty': x_pred_original[:, 1],
    'tz': x_pred_original[:, 2]
})
reconstructed_df.to_csv('../data/processed/uav_reconstructed_trajectory.csv', index=False)
print("Reconstructed trajectory saved to 'uav_reconstructed_trajectory.csv'")

Reconstructed trajectory saved to 'uav_reconstructed_trajectory.csv'
